# CW-Husky SAD Stress Test

Checks that:
1. The FPGA doesn't overheat when SAD is run continuously at its maximum clock frequency;
2. SAD captures are successful at the maximum clock frequency.

(1) doesn't need any specific target (or any target at all, really).

(2) needs a specific target (SAM4S with the firmware that we program here) in order for the SAD captures to work out-of-the-box, without any tweaking.

**If at any time the connection with Husky is "lost", the test has failed, and this needs to be investigated.**

**If any `scope.XADC` errors occur, they need to be investigated.**

In [ ]:
PLATFORM = 'CW308_SAM4S'
SS_VER = "SS_VER_1_1"

In [ ]:
import chipwhisperer as cw
scope = cw.scope()

In [ ]:
if scope._is_husky_plus:
    MAXFREQ = 250e6
else:
    MAXFREQ = 200e6

In [ ]:
%run ../jupyter/Setup_Scripts/Setup_Generic.ipynb

In [ ]:
scope.default_setup()

In [ ]:
cw.program_target(scope, prog, "../../firmware/mcu/simpleserial-trace/simpleserial-trace-{}.hex".format(PLATFORM))
reset_target(scope)

Change target clock to 10 MHz so that we can hit `MAXFREQ` on the nose:

In [ ]:
scope.clock.clkgen_freq = 10e6
reset_target(scope)
target.baud = 38400 * 10/7.37

Check target is alive:

In [ ]:
scope.trigger.module = 'basic'
scope.trigger.triggers = 'tio4'

# these are not the errors we care about in this test:
scope.adc.lo_gain_errors_disabled = True
scope.adc.clip_errors_disabled = True

scope.adc.samples = 35000
scope.adc.presamples = 0
scope.adc.segments = 1
scope.adc.bits_per_sample = 8  # SAD is done at 8 bits per sample

scope.gain.db = 10

reftrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16), as_int=True)
assert scope.adc.trig_count == 31864, "Unexpected trigger count. Are you running the correct firmware?"

Crank up the ADC clock to its maximum allowed value:

In [ ]:
scope.clock.adc_mul = int(MAXFREQ / scope.clock.clkgen_freq)
reset_target(scope)
assert abs(scope.clock.adc_freq - MAXFREQ)/scope.clock.adc_freq < 0.01

Check that basic capture still works:

In [ ]:
reftrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16), as_int=True)

Set up SAD:

In [ ]:
refstart = 1000
scope.SAD.reference = reftrace.wave[refstart:]
scope.SAD.threshold = 20
scope.SAD.interval_threshold = 20
scope.SAD.multiple_triggers = True
scope.SAD.emode = False
scope.SAD.always_armed = False

scope.trigger.module = 'SAD'

# 🔥 Let's get hot 🔥

In this first test we set `scope.SAD.always_armed`, which turns on all the SAD logic (even if we're not actively trying to trigger). No target is required for this.

We continuously poll the FPGA temperature (and plot it); we periodically do a least squares linear regression on the last minute of temperature data; we stop when the slope is close enough to flat.

This takes a few minutes.

In [ ]:
scope.SAD.always_armed = True

In [ ]:
import numpy as np
import scipy.stats
from ipywidgets import interact, Layout
from bokeh.io import push_notebook, output_notebook
from bokeh.models import Span, Legend, LegendItem
from bokeh.plotting import figure, show

def update_plot():
    temps.append(scope.XADC.temp)
    S1.data_source.data['x'] = list(range(len(temps)))
    S1.data_source.data['y'] = temps
    push_notebook()

output_notebook()
S = figure(width=1800)
temps = [scope.XADC.temp]
xrange = [0]
S1 = S.line(xrange, temps, line_color='red')
show(S, notebook_handle=True)

In [ ]:
CHECK_INTERVAL = 60 # check slope every this many seconds
TEMP_INTERVAL = 0.5 # measure temperature every this many seconds
MIN_RUN_TIME = 300 # run for at least this many seconds
print('Long test running, check plot above, temperature [celcius] as a function of time [seconds] to see the temperature rising... results checked every %d seconds' % CHECK_INTERVAL)
while True:
    measurements = int(CHECK_INTERVAL/TEMP_INTERVAL)
    for i in range(measurements):
        update_plot()
        time.sleep(TEMP_INTERVAL)
    slope = scipy.stats.linregress(range(measurements), temps[-measurements:]).slope
    predicted_increase = slope*measurements
    seconds_elapsed = TEMP_INTERVAL*len(temps)
    if scope.XADC.status != 'good':
        print('❌ XADC error detected! This should not happen. %s' % scope.XADC.errors)
        scope.SAD.always_armed = False
        break
    if predicted_increase < 0.1 and seconds_elapsed > MIN_RUN_TIME: # stop when the slope predicts a < 0.1C increase over the next set of measurements
        print('✅ Temperature looks stable! slope for last chunk of measurements: %0.5f; max temp: %3.1f; average max temp: %3.1f' % (slope, max(temps), np.average(temps[-measurements:])))
        scope.SAD.always_armed = False
        break
    else:
        print('🔥 Temperature still increasing; slope for last chunk of measurements: %0.5f; max temp: %3.1f; predicted increase: %3.1f' % (slope, max(temps), predicted_increase))

In the above we carefully monitored the temperature; now let's check the min/max voltages seen on the FPGA's VCC rails:

In [ ]:
check_vcc_rails()

In [ ]:
def check_vcc_rails():
    failed = False
    for rail, nominal in zip(['vccint', 'vccaux', 'vccbram'],  [1.0, 1.8, 1.0]):
        for worst,limit in zip(['min', 'max'], ['lower', 'upper']):
            vseen = scope.XADC.get_vcc(rail, worst)
            vlimit = scope.XADC._get_vcc_limit(rail, limit)
            if worst == 'min':
                vmargin = vseen - vlimit
            else:
                vmargin = vlimit - vseen
            if vmargin > 0:
                status = '✅ pass'
            else:
                status = '❌ FAIL!'
                failed = True
            print('%7s: nominal: %1.2f, %s seen: %1.2f, limit: %1.2f, margin: %1.2f   %s' % (rail, nominal, worst, vseen, vlimit, vmargin, status))
    assert not failed


# Now let's gets the temperature even higher:

We run a capture that will fail to trigger with a very long `scope.adc.timeout`. This *may* drive the temperature higher still.

This part of the test takes 3 minutes.

"No trigger seen" and "Timeout happened during capture" warnings are normal and expected here.

In [ ]:
scope.SAD.always_armed = True
scope.SAD.interval_threshold = 2
scope.adc.timeout = 30

In [ ]:
for i in range(6):
    sadtrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16), as_int=True)
    assert scope.XADC.status == 'good'
    print('Iteration %d/5: temp = %3.1f' % (i, scope.XADC.get_temp()))
print('✅ Success!')

In [ ]:
scope.SAD.always_armed = False

Check the VCC rails again:

In [ ]:
check_vcc_rails()

# Run actual SAD captures at the maximum ADC frequency

This requires the expected target and firmware to run out-of-the box.

In [ ]:
scope.adc.timeout = 2
scope.adc.stream_mode = False
scope.adc.samples = 98000
scope.adc.presamples = 0
scope.adc.segments = 1
scope.gain.db = 20
scope.trigger.module = 'basic'

reftrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16), as_int=True)

assert not scope.adc.errors

In [ ]:
if scope._is_husky_plus:
    refstart = 55550
else:
    # different ADC frequency = different offset!
    refstart = int(55550/25*20)
scope.SAD.emode = True
scope.SAD.reference = reftrace.wave[refstart:]

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Span

output_notebook()
p = figure(width=1800, tools='pan, box_zoom, hover, reset, save')

xrange = list(range(len(reftrace.wave)))
p.line(xrange, reftrace.wave)
p.renderers.extend([Span(location=refstart, dimension='height', line_color='black', line_width=2)])
p.renderers.extend([Span(location=refstart+scope.SAD.sad_reference_length, dimension='height', line_color='black', line_width=2)])

show(p)

In [ ]:
scope.trigger.module = 'SAD'

scope.SAD.always_armed = False
scope.SAD.multiple_triggers = True
scope.SAD.interval_threshold = 10
scope.SAD.threshold = 10

scope.adc.stream_mode = False
scope.adc.samples = 3000
scope.adc.presamples = scope.SAD.sad_reference_length + scope.SAD.latency
scope.adc.segments = 10

Try once...

In [ ]:
scope.SAD.reference = reftrace.wave[refstart:]

In [ ]:
sadtrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16), as_int=True)
assert scope.SAD.num_triggers_seen == 10
assert scope.XADC.status == 'good'
assert not scope.adc.errors

Then a few more times:

In [ ]:
scope.SAD.always_armed = False

In [ ]:
from tqdm.notebook import tnrange
for i in tnrange(100):
    sadtrace = cw.capture_trace(scope, target, bytearray(16), bytearray(16), as_int=True)
    assert scope.SAD.num_triggers_seen == 10
    assert scope.XADC.status == 'good'
    assert not scope.adc.errors

Visual check:

In [ ]:
from bokeh.palettes import inferno
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import itertools

SAMPLES = scope.SAD.sad_reference_length

numplots = scope.adc.segments
xrange = list(range(SAMPLES))
p = figure(width=1800)
colors = itertools.cycle(inferno(numplots))
for i in range(numplots):
    offset = i*scope.adc.samples
    p.line(xrange, sadtrace.wave[offset:offset+SAMPLES], color=next(colors))

p.line(xrange, scope.SAD.reference[:SAMPLES], line_color='grey', line_width=3, line_dash='dotted')
show(p)

In [ ]:
assert scope.XADC.status == 'good'

This in particular can stress the VCC rails:

In [ ]:
check_vcc_rails()

**If everything until here passed, the test is done!**

If SAD triggering doesn't work as expected, try tweaking the parameters with the help of `SADEXplorer`. (There is no need to run this if everything above passed without errors.)

In [ ]:
#explorer = cw.SADExplorer(scope, target, reftrace.wave, refstart, max_segments=10)

Otherwise, turn off all the hot stuff:

In [ ]:
scope.trigger.module = 'basic'
scope.clock.adc_mul = 1